In [ ]:
from bs4 import BeautifulSoup
import requests
import jieba
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import jieba.analyse as analyse

#模擬使用者讀取網頁
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.92 Safari/537.36'}

# 目標網站
url = 'https://www.pttweb.cc/hot/news/today'
# 發出 get請求
resp = requests.get(url, headers=headers)
# 網頁編碼
resp.encoding = 'utf-8'
# 進行網頁解析
soup = BeautifulSoup(resp.text, 'html.parser')
# 找出所有 div區塊中 帶有目標class名的元素
divs = soup.find_all('div', 'e7-right-top-container e7-no-outline-all-descendants')
# 新增個list存取爬下來的內容
articles = []
# 換個網頁爬文章內容
root = 'https://www.ptt.cc'
for div in divs:
    link = div.find('a')['href']
    title = div.find('span', 'e7-show-if-device-is-not-xs').text
    articles.append({
        'title':title.replace('\u3000', ' '),
        'link':root + link + '.html'
    })
for article in articles:
    res = requests.get(article['link'], headers = {'cookie': 'over18=1;'})
    if res.status_code == 404:
        articles.remove(article)
    else:
        soup = BeautifulSoup(res.text, 'lxml')
        main = soup.find('div', id='main-content')
        main_tag = main.find_all('div', class_='push')
        # 新增一個位置存取文章的推文內容
        comment = []

        for i in main_tag:
            # 如果沒有推文就跳過
            if not i.find('span', 'push-tag'):
                continue
            # 把: 標點符號替換掉    
            push_content = i.find('span', 'push-content').text.replace(': ',"")
            # 推文內容存入
            comment.append(push_content)
    
        article['content'] = comment

jieba.set_dictionary('dict.txt.big.txt')

# 抓前十篇熱門新聞製作文字雲
for article in articles[:10]:
    content_text = article['content']
    content_list = " ".join(content_text)
    tfidf_fre = jieba.analyse.extract_tags(content_list, topK=100, withWeight=True, allowPOS=(),withFlag=True)
    
    # 把分析完的詞頻輸出成字典
    count_dic = {}
    for i in range(len(tfidf_fre)):
        count_dic[tfidf_fre[i][0]] = tfidf_fre[i][1]
    # 把字典交給wordcloud做成文字雲
    myWordClode = WordCloud(
                       width=1200, # 圖的寬度
                       height=600,      # 圖的長度
                       background_color="black",  # 背景顏色 預設是白色
                       colormap="Dark2",
                       font_path='SourceHanSansTW-Regular.otf' # 輸出字體 必須將字體的檔案放在同個資料夾下
                        ).fit_words(count_dic)
    # 用PIL顯示文字雲
    plt.figure(figsize=(8, 6), dpi=100) 
    plt.imshow(myWordClode)
    plt.axis("off")
    print(article['title'])
    print(article['link'])
    plt.show()